In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = c3e1ce5
molecule = {'co2': 0.0008}
band = [3]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.905493                    1  0.624204
0.000750        2  0.889631                    2  0.630223
0.001052        3  0.905648                    3  0.651046
0.001476        4  0.917695                    4  0.672307
0.002070        5  0.922694                    5  0.691495
0.002904        6  0.918526                    6  0.707948
0.004074        7  0.902787                    7  0.719789
0.005714        8  0.873344                    8  0.725496
0.008015        9  0.827903                    9  0.721872
0.011243       10  0.764659                   10  0.705967
0.015771       11  0.682561                   11  0.672823
0.022122       12  0.581440                   12  0.616617
0.031031       13  0.461500                   13  0.531467
0.043528       14  0.321498                   14  0.411880
0.061057       15  0.214035                   15  0.303756
0.085645       16  0.225817                   16  0.285348
0.120136       17  0.308857                   17  0.319031
0.168516       18  0.370060                   18  0.327917
0.236378       19  0.429327                   19  0.349137
0.331549       20  0.511612                   20  0.428596
0.465100       21  0.636112                   21  0.577545
0.652400       22  0.839767                   22  0.806243
0.915100       23  1.068454                   23  1.028362
1.283650       24  1.033069                   24  0.989097
1.800600       25  0.800511                   25  0.778236
2.525700       26  0.614690                   26  0.618402
3.542800       27  0.477670                   27  0.505622
4.969550       28  0.372308                   28  0.404413
6.970850       29  0.293714                   29  0.319083
9.778100       30  0.233196                   30  0.247413
13.715850      31  0.180021                   31  0.184768
19.239350      32  0.132694                   32  0.128864
26.987250      33  0.101568                   33  0.100423
37.855300      34  0.084028                   34  0.094033
53.100050      35  0.066044                   35  0.082838
73.887500      36  0.046401                   36  0.062480
97.662500      37  0.034456                   37  0.046995
121.437500     38  0.028242                   38  0.038303
145.212500     39  0.015961                   39  0.024072
168.987500     40  0.001949                   40  0.008858
192.762500     41  0.004258                   41  0.012086
216.537500     42  0.022837                   42  0.033480
240.312500     43  0.038905                   43  0.052007
264.087500     44  0.051574                   44  0.066792
287.862500     45  0.062315                   45  0.080750
311.637500     46  0.071595                   46  0.419673
335.412500     47  0.079664                   47  0.464654
359.187500     48  0.086693                   48  0.074539
382.962500     49  0.092814                   49  0.166625
406.737500     50  0.098048                   50  0.091585
430.512500     51  0.102355                   51  1.308280
454.287500     52  0.105631                   52 -0.376054
478.062500     53  0.107873                   53  0.018861
501.837500     54  0.109283                   54  0.027098
525.612500     55  0.110162                   55  0.033490
549.387500     56  0.110751                   56  0.038998
573.162500     57  0.111188                   57  0.043928
596.937500     58  0.111564                   58  0.048439
620.712500     59  0.111943                   59  0.052639
644.487500     60  0.112372                   60  0.056591
668.262500     61  0.112902                   61  0.060439
692.037500     62  0.113619                   62  0.064253
715.812500     63  0.114621                   63  0.068432
739.587500     64  0.116019                   64  0.073013
763.362500     65  0.117928                   65  0.078210
78

# Fluxes by Layer

CRD                       CLIRAD             \
                        flug       fldg      fnetg      i       flug   
pressure    level                                                      
0.000000    1     -27.975271   0.000000 -27.975271     11 -22.764850   
0.000624    2     -27.975295   0.000091 -27.975204     11 -22.764873   
0.000876    3     -27.975305   0.000127 -27.975178     11 -22.764882   
0.001229    4     -27.975318   0.000178 -27.975140     11 -22.764894   
0.001723    5     -27.975336   0.000250 -27.975086     11 -22.764911   
0.002417    6     -27.975361   0.000351 -27.975010     11 -22.764935   
0.003391    7     -27.975395   0.000490 -27.974904     11 -22.764968   
0.004757    8     -27.975441   0.000683 -27.974758     11 -22.765013   
0.006672    9     -27.975505   0.000945 -27.974560     11 -22.765076   
0.009359    10    -27.975594   0.001297 -27.974297     11 -22.765163   
0.013128    11    -27.975717   0.001762 -27.973955     11 -22.765285   
0.018415    12    -27.975890   0.002362 -27.973528     11 -22.765457   
0.025830    13    -27.976138   0.003121 -27.973017     11 -22.765705   
0.036232    14    -27.976503   0.004055 -27.972448     11 -22.766070   
0.050823    15    -27.977068   0.005176 -27.971893     11 -22.766627   
0.071291    16    -27.977957   0.006583 -27.971374     11 -22.767481   
0.100000    17    -27.979254   0.008648 -27.970606     11 -22.768695   
0.140271    18    -27.980985   0.011852 -27.969132     11 -22.770283   
0.196760    19    -27.983224   0.016568 -27.966656     11 -22.772291   
0.275997    20    -27.986011   0.023385 -27.962626     11 -22.774724   
0.387100    21    -27.989249   0.033356 -27.955893     11 -22.777451   
0.543100    22    -27.992447   0.048308 -27.944138     11 -22.779957   
0.761700    23    -27.993565   0.071173 -27.922393     11 -22.780344   
1.068500    24    -27.987941   0.104378 -27.883562     11 -22.774631   
1.498800    25    -27.974541   0.143636 -27.830905     11 -22.761973   
2.102400    26    -27.957378   0.183711 -27.773667     11 -22.745599   
2.949000    27    -27.937934   0.225911 -27.712023     11 -22.726677   
4.136600    28    -27.916655   0.271831 -27.644824     11 -22.705319   
5.802500    29    -27.894050   0.322696 -27.571354     11 -22.681797   
8.139200    30    -27.870215   0.380161 -27.490054     11 -22.655781   
11.417000   31    -27.845113   0.445603 -27.399509     11 -22.627023   
16.014700   32    -27.820489   0.519024 -27.301465     11 -22.597030   
22.464000   33    -27.801343   0.601252 -27.200091     11 -22.571213   
31.510500   34    -27.791284   0.700036 -27.091249     11 -22.553342   
44.200100   35    -27.791273   0.826332 -26.964940     11 -22.543578   
62.000000   36    -27.810691   0.985006 -26.825685     11 -22.550699   
85.775000   37    -27.869931   1.174927 -26.695004     11 -22.595279   
109.550000  38    -27.956869   1.358903 -26.597967     11 -22.668148   
133.325000  39    -28.064680   1.546252 -26.518428     11 -22.761738   
157.100000  40    -28.203208   1.729732 -26.473476     11 -22.886657   
180.875000  41    -28.388388   1.920401 -26.467987     11 -23.059467   
204.650000  42    -28.613894   2.157899 -26.455996     11 -23.269147   
228.425000  43    -28.857710   2.466031 -26.391679     11 -23.490960   
252.200000  44    -29.112241   2.830129 -26.282111     11 -23.714858   
275.975000  45    -29.375186   3.238323 -26.136863     11 -23.936792   
299.750000  46    -29.644829   3.683466 -25.961363     11 -24.152119   
323.525000  47    -29.919633   4.159904 -25.759728     11 -24.759268   
347.300000  48    -30.198161   4.662791 -25.535369     11 -25.965138   
371.075000  49    -30.479030   5.187815 -25.291215     11 -26.184286   
394.850000  50    -30.760863   5.731044 -25.029820     11 -26.832746   
418.625000  51    -31.042221   6.288537 -24.753684     11 -27.030330   
442.400000  52    -31.321974   6.856555 -24.465419     11 -29.698431   
466.175000  53    -31.599454   7.431525 -24.167929     11 -31.315

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')